In [6]:
pip install torch_geometric

In [7]:
pip install torch torchvision torchaudio

In [4]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 33.9 MB/s eta 0:00:00


In [1]:
import os
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import global_add_pool
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem, rdchem
import matplotlib.pyplot as plt

환경결정상수

In [2]:
# H5 파일 키 설정
KEY_CONFIG = {
    "atoms": "elements", "smiles": "Rsmiles",
    "R_coords": "RG", "TS_coords": "TSG", "P_coords": "PG",          
    "R_energy": { "H": "R_H", "G": "R_F", "E": "R_E" },
    "TS_energy": { "H": "TS_H", "G": "TS_F", "E": "TS_E" },
    "P_energy": { "H": "P_H", "G": "P_F", "E": "P_E" }
}

# 결합 해리 에너지 (BDE) 테이블 (kcal/mol)
BDE_TABLE = {
    (1, 1): 104, (1, 6): 99, (1, 7): 93, (1, 8): 111, (1, 9): 135,
    (6, 6): 83, (6, 7): 73, (6, 8): 86, (6, 9): 116,
    (7, 7): 39, (7, 8): 53, (8, 8): 35
}

# 원자 물성 [EN, Hardness, Radius, IE]
ATOM_PROPS_TABLE = {
    1: [2.20, 6.42, 0.31, 13.60], 6: [2.55, 5.00, 0.76, 11.26],
    7: [3.04, 7.27, 0.71, 14.53], 8: [3.44, 6.08, 0.66, 13.62], 9: [3.98, 7.01, 0.57, 17.42]
}
DEFAULT_PROPS = [0.0, 0.0, 0.0, 0.0]

# 작용기 SMARTS 패턴
FUNCTIONAL_GROUPS = {
    "Nitro": "[N+](=O)[O-]", "Sulfonyl": "S(=O)(=O)", "Cyano": "C#N", "Trifluoromethyl": "C(F)(F)F",
    "Carbonyl": "[CX3]=[OX1]", "Carboxyl": "[CX3](=O)[OX2H1]", "Halogen": "[F,Cl,Br,I]",
    "Amine": "[NX3;H2,H1,H0;!$(NC=O)]", "Hydroxy": "[OX2H]", "Alkoxy": "[OX2][CX4]", 
    "Amide": "[NX3][CX3](=O)", "Alkyl": "[CX4]", "Phenyl": "c1ccccc1"
}
FG_TYPE_MAP = {
    "Nitro": -1.0, "Sulfonyl": -1.0, "Cyano": -1.0, "Trifluoromethyl": -1.0,
    "Carbonyl": -0.5, "Carboxyl": -0.5, "Halogen": -0.2, "Amine": 1.0, "Hydroxy": 1.0, 
    "Alkoxy": 1.0, "Amide": 0.2, "Alkyl": 0.3, "Phenyl": 0.1
}

데이터셋

In [3]:
class ReactionDataset(Dataset):
    def __init__(self, h5_path, limit=None):
        super().__init__(None, transform=None, pre_transform=None)
        self.h5_path = h5_path
        if not os.path.exists(h5_path): raise FileNotFoundError(f"File not found: {h5_path}")
        with h5py.File(self.h5_path, 'r') as f:
            self.keys = list(f.keys())
            if limit: self.keys = self.keys[:limit]
            self.len_data = len(self.keys)
        self.fg_patterns = {name: Chem.MolFromSmarts(smarts) for name, smarts in FUNCTIONAL_GROUPS.items()}

    def len(self): return self.len_data

    def get_atom_features(self, mol):
        try: AllChem.ComputeGasteigerCharges(mol); 
        except:
            pass
        features, hsab_list = [], []
        
        # 작용기(Functional Group) 탐지
        atom_fg_info = {atom.GetIdx(): {'ewg':0.0, 'edg':0.0, 'types':[]} for atom in mol.GetAtoms()}
        for name, pattern in self.fg_patterns.items():
            if pattern:
                for match in mol.GetSubstructMatches(pattern):
                    effect = FG_TYPE_MAP.get(name, 0.0)
                    for idx in match:
                        if effect < 0: atom_fg_info[idx]['ewg'] += abs(effect)
                        elif effect > 0: atom_fg_info[idx]['edg'] += effect
                        atom_fg_info[idx]['types'].append(name)
        
        atoms_with_map = [(a.GetAtomMapNum() if a.GetAtomMapNum()!=0 else a.GetIdx()+1, a) for a in mol.GetAtoms()]
        atoms_with_map.sort(key=lambda x: x[0])
        
        for _, atom in atoms_with_map:
            feat = []
            z = atom.GetAtomicNum()
            idx = atom.GetIdx()
            
            # [1] Basic Info (21 dims)
            sym_map={'H':0,'C':1,'N':2,'O':3,'F':4}; oh=[0]*6; oh[sym_map.get(atom.GetSymbol(),5)]=1; feat.extend(oh)
            hyb=atom.GetHybridization(); hyb_map={rdchem.HybridizationType.SP:0, rdchem.HybridizationType.SP2:1, rdchem.HybridizationType.SP3:2, rdchem.HybridizationType.SP3D:3, rdchem.HybridizationType.SP3D2:4}; hyb_vec=[0]*5; 
            if hyb in hyb_map: hyb_vec[hyb_map[hyb]]=1
            feat.extend(hyb_vec)
            try: q=float(atom.GetProp('_GasteigerCharge')); 
            except: q=0.0
            if np.isnan(q): q=0.0
            feat.extend([q, float(atom.GetFormalCharge()), float(atom.GetTotalValence()), atom.GetMass()/20.0])
            feat.extend([1.0 if atom.IsInRingSize(s) else 0.0 for s in [3,4,5,6]])
            feat.extend([1.0 if atom.GetIsAromatic() else 0.0, 1.0 if atom.GetChiralTag()!=rdchem.ChiralType.CHI_UNSPECIFIED else 0.0])
            
            # [2] Functional Group Info (6 dims)
            info = atom_fg_info[idx]
            feat.extend([info['ewg'], info['edg'], 1.0 if 'Nitro' in info['types'] or 'Sulfonyl' in info['types'] else 0.0, 1.0 if 'Carbonyl' in info['types'] else 0.0, 1.0 if 'Amine' in info['types'] or 'Hydroxy' in info['types'] else 0.0, 1.0 if 'Halogen' in info['types'] else 0.0])
            features.append(feat)
            
            # [3] HSAB Props
            props = ATOM_PROPS_TABLE.get(z, DEFAULT_PROPS)
            hsab_list.append([-props[0], props[1]]) # [Chemical Potential, Hardness]
            
        return torch.tensor(features, dtype=torch.float), torch.tensor(hsab_list, dtype=torch.float)

    def get(self, idx):
        with h5py.File(self.h5_path, 'r') as f:
            group = f[self.keys[idx]]
            
            # RDKit Parsing
            if KEY_CONFIG["smiles"] in group:
                smi = group[KEY_CONFIG["smiles"]][()]; smi = smi.decode('utf-8') if isinstance(smi, bytes) else str(smi)
                mol = Chem.MolFromSmiles(smi)
                if mol: mol = Chem.AddHs(mol); x, hsab = self.get_atom_features(mol)
                else: atoms = group[KEY_CONFIG["atoms"]][()]; x = torch.zeros(len(atoms), 27); hsab = torch.zeros(len(atoms), 2)
            else: atoms = group[KEY_CONFIG["atoms"]][()]; x = torch.zeros(len(atoms), 27); hsab = torch.zeros(len(atoms), 2)
            
            # Coordinates
            pos_R = torch.tensor(group[KEY_CONFIG["R_coords"]][()], dtype=torch.float)
            pos_TS = torch.tensor(group[KEY_CONFIG["TS_coords"]][()], dtype=torch.float)
            pos_P = torch.tensor(group[KEY_CONFIG["P_coords"]][()], dtype=torch.float)
            
            # Energies
            e_R = torch.tensor([group[KEY_CONFIG["R_energy"][k]][()] for k in ["H","G","E"]], dtype=torch.float)
            e_TS = torch.tensor([group[KEY_CONFIG["TS_energy"][k]][()] for k in ["H","G","E"]], dtype=torch.float)
            e_P = torch.tensor([group[KEY_CONFIG["P_energy"][k]][()] for k in ["H","G","E"]], dtype=torch.float)
            
            # Targets (Delta Learning ready: predicting barriers directly)
            y_fwd = (e_TS - e_R).unsqueeze(0)
            y_rev = (e_TS - e_P).unsqueeze(0)
            thermo = (e_P - e_R).unsqueeze(0)
            
            # Consistency Check
            if x.size(0)!=pos_R.size(0): m=min(x.size(0),pos_R.size(0)); x=x[:m]; hsab=hsab[:m]; pos_R=pos_R[:m]; pos_P=pos_P[:m]; pos_TS=pos_TS[:m]
            
            # Fully Connected Edges
            n=x.size(0); r=torch.arange(n).repeat_interleave(n); c=torch.arange(n).repeat(n); mask=r!=c
            edge_index=torch.stack([r[mask], c[mask]], dim=0)
            
            # BDE Features
            sym=torch.argmax(x[:,:6], dim=1); map_z={0:1,1:6,2:7,3:8,4:9,5:0}; z=torch.tensor([map_z.get(i.item(),0) for i in sym], dtype=torch.long)
            bde = [BDE_TABLE.get(tuple(sorted((int(z[i]), int(z[j])))), 80.0) for i,j in edge_index.T]
            edge_bde = torch.tensor(bde, dtype=torch.float).unsqueeze(-1)
            
            return Data(x=x, z=z, hsab=hsab, pos=pos_R, pos_P=pos_P, edge_index=edge_index, 
                        y_fwd=y_fwd, y_rev=y_rev, y_pos=pos_TS, thermo_energy=thermo, edge_bde=edge_bde)

GNN Backbone

In [4]:
class RBFLayer(nn.Module):
    """
    [Radial Basis Function Layer]
    거리 정보를 단순히 하나의 숫자가 아닌, 여러 개의 기저 함수(Gaussian)로 확장하여 표현합니다.
    이는 Geometric Deep Learning (SchNet, DimeNet)의 핵심 요소입니다.
    """
    def __init__(self, num_rbf=50, cutoff=5.0):
        super().__init__()
        self.cutoff = cutoff
        # 0 ~ cutoff 사이를 num_rbf 개수만큼 등분
        self.centers = nn.Parameter(torch.linspace(0, cutoff, num_rbf), requires_grad=False)
        self.widths = nn.Parameter(torch.tensor([cutoff / num_rbf] * num_rbf), requires_grad=False)

    def forward(self, dist):
        # Gaussian Smearing: exp(- (d - mu)^2 / beta)
        return torch.exp(-((dist - self.centers) ** 2) / self.widths)

In [5]:
class GeometricInteractionLayer(nn.Module):
    """
    [Geometric Interaction Block]
    RBF로 확장된 거리 정보를 이용하여 노드 간의 상호작용을 학습합니다.
    Simple MLP보다 훨씬 풍부한 기하학적 특징을 추출합니다.
    """
    def __init__(self, hidden_dim, num_rbf=50):
        super().__init__()
        self.rbf_lin = nn.Linear(num_rbf, hidden_dim)
        
        # Message Passing MLPs
        self.msg_net = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.update_net = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, h, edge_index, rbf_dist):
        row, col = edge_index
        
        # 1. 거리 정보 변환 (RBF -> Hidden)
        edge_geom = self.rbf_lin(rbf_dist) # [E, Hidden]
        
        # 2. 메시지 생성 (Source + Target) * Geometry
        # SchNet 방식의 Continuous Filter Convolution 아이디어 차용
        node_feat = torch.cat([h[row], h[col]], dim=-1)
        msg_raw = self.msg_net(node_feat)
        msg = msg_raw * edge_geom # Element-wise multiplication with Geometry
        
        # 3. 집계 (Aggregation)
        aggr = torch.zeros_like(h)
        aggr.index_add_(0, row, msg)
        
        # 4. 업데이트 (Update)
        update = self.update_net(torch.cat([h, aggr], dim=-1))
        
        return self.norm(h + update)

In [6]:
class GeometricGNN(nn.Module):
    """
    [Main Backbone]
    위상학적 정보뿐만 아니라 기하학적(RBF) 정보를 적극 활용하는 GNN입니다.
    """
    def __init__(self, node_dim, hidden_dim, num_layers=4):
        super().__init__()
        self.node_emb = nn.Linear(node_dim, hidden_dim)
        self.rbf = RBFLayer(num_rbf=50, cutoff=10.0)
        self.layers = nn.ModuleList([
            GeometricInteractionLayer(hidden_dim, num_rbf=50) for _ in range(num_layers)
        ])

    def forward(self, x, pos, edge_index):
        h = self.node_emb(x)
        
        # 거리 계산 및 RBF 변환
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1, keepdim=True)
        rbf_feat = self.rbf(dist) # [E, 50]
        
        for layer in self.layers:
            h = layer(h, edge_index, rbf_feat)
            
        return h

In [7]:
class SigmoidGating(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.gate_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 1, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 1), nn.Sigmoid()
        )
    def forward(self, h, edge_index, pos):
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1, keepdim=True)
        return self.gate_mlp(torch.cat([h[row], h[col], dist], dim=-1))

In [8]:
class StructureDecoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(nn.Linear(hidden_dim*2, hidden_dim), nn.SiLU(), nn.Linear(hidden_dim, 1))
    def forward(self, h, edge_index, gate_score):
        row, col = edge_index
        return self.mlp(torch.cat([h[row], h[col]], dim=-1)) * gate_score

물리법칙

In [9]:
class PaulingBondLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.r0_table = torch.tensor([0.0, 0.31, 0.0, 0.0, 0.0, 0.0, 0.76, 0.71, 0.66, 0.57]) 
    def forward(self, z, edge_index, dist):
        row, col = edge_index
        r0_i = self.r0_table.to(z.device)[z[row]]
        r0_j = self.r0_table.to(z.device)[z[col]]
        r0_sum = (r0_i + r0_j).unsqueeze(-1)
        return torch.exp((r0_sum - dist) / 0.3)

In [10]:
class StereoelectronicLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, pos_R, pos_P, edge_index):
        row, col = edge_index
        vec_R = F.normalize(pos_R[col] - pos_R[row], p=2, dim=-1)
        vec_P = F.normalize(pos_P[col] - pos_P[row], p=2, dim=-1)
        return (vec_R * vec_P).sum(dim=-1, keepdim=True)

In [11]:
class HSABInteractionLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, hsab, edge_index):
        row, col = edge_index
        mu, eta = hsab[:, 0], hsab[:, 1]
        d_mu = (mu[row] - mu[col]).pow(2)
        sum_eta = eta[row] + eta[col]
        return (d_mu / (sum_eta + 1e-6)).unsqueeze(-1)

In [12]:
class StericLayer(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, pos, edge_index):
        row, col = edge_index
        dist = (pos[row] - pos[col]).norm(dim=-1)
        density = 1.0 / (dist.pow(2) + 0.1)
        steric = torch.zeros(pos.size(0), 1, device=pos.device)
        steric.index_add_(0, row, density.unsqueeze(-1))
        return steric

In [13]:
class HammondGate(nn.Module):
    def __init__(self):
        super().__init__()
        self.gate = nn.Sequential(nn.Linear(3, 16), nn.SiLU(), nn.Linear(16, 1), nn.Sigmoid())
    def forward(self, thermo): return self.gate(thermo)

In [14]:
class AdaptiveBEPLayer(nn.Module):
    """
    [Bell-Evans-Polanyi Principle]
    E_a = alpha * Delta_H + beta
    분자 그래프 전체의 특징(h_graph)을 기반으로 반응별 alpha와 beta를 예측합니다.
    """
    def __init__(self, hidden_dim):
        super().__init__()
        self.coef_net = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.SiLU(),
            nn.Linear(hidden_dim // 2, 2) # [alpha, beta]
        )

    def forward(self, h_graph, delta_H):
        # h_graph: [Batch, Hidden]
        # delta_H: [Batch, 1]
        coefs = self.coef_net(h_graph)
        
        # alpha: 0~1 사이의 기울기 (일반적인 화학 반응 범위)
        alpha = torch.sigmoid(coefs[:, 0:1]) 
        # beta: 기본 에너지 장벽 (항상 양수)
        beta = F.softplus(coefs[:, 1:2])
        
        return alpha * delta_H + beta

Predictor

In [15]:
class PhysChemInformedPredictor(nn.Module):
    def __init__(self, hidden_dim, dropout=0.1):
        super().__init__()
        # 기존 4대 물리 레이어 (변경 없음)
        self.pauling = PaulingBondLayer()
        self.stereo = StereoelectronicLayer()
        self.hsab = HSABInteractionLayer()
        self.steric = StericLayer() # (기존 단순 Steric 유지)
        self.hammond = HammondGate()
        
        # [추가됨] BEP Layer
        self.bep_layer = AdaptiveBEPLayer(hidden_dim)
        
        # Node MLP
        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_dim + 4, hidden_dim), 
            nn.SiLU(), nn.Dropout(dropout), 
            nn.Linear(hidden_dim, 3)
        )
        
        # Edge MLP (Residual 예측용)
        edge_in = (hidden_dim * 2) + 11
        self.edge_mlp = nn.Sequential(
            nn.Linear(edge_in, hidden_dim), nn.SiLU(), nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 6) 
        )
        
        # 잔차(Residual) 계산용 헤드
        # BEP가 예측한 기준값에서 얼마나 보정해야 하는지(Residual)를 학습
        self.residual_head = nn.Linear(6, 1) 

    def forward(self, h, z, hsab, edge_index, pos_R, pos_P, gate_score, pred_delta_D, batch, thermo, bde):
        row, col = edge_index
        late_char = self.hammond(thermo); late_char_edge = late_char[batch[row]]
        
        # 1. Graph Level Pooling for BEP
        h_graph = global_add_pool(h, batch)
        
        # 2. BEP Base Energy Calculation
        # thermo[:, 0:1]은 Enthalpy(Delta H)를 의미
        delta_H = thermo[:, 0:1]
        E_bep_base = self.bep_layer(h_graph, delta_H)
        
        # 3. Existing Physics Features
        steric_R = self.steric(pos_R, edge_index)
        steric_graph = global_add_pool(steric_R, batch)
        base_node_E = self.node_mlp(torch.cat([h_graph, thermo, steric_graph], dim=-1))
        
        dist_R = (pos_R[row]-pos_R[col]).norm(dim=-1, keepdim=True)
        dist_P = (pos_P[row]-pos_P[col]).norm(dim=-1, keepdim=True)
        bo_R = self.pauling(z, edge_index, dist_R)
        bo_P = self.pauling(z, edge_index, dist_P)
        bo_TS = (1.0 - late_char_edge) * bo_R + late_char_edge * bo_P
        align = self.stereo(pos_R, pos_P, edge_index)
        hsab_E = self.hsab(hsab, edge_index)
        
        edge_feat = torch.cat([
            h[row], h[col], dist_R, dist_P, gate_score,
            bo_R, bo_P, bo_TS, bde, align, late_char_edge, hsab_E, pred_delta_D
        ], dim=-1)
        
        # 4. Residual Prediction
        edge_out = self.edge_mlp(edge_feat) * gate_score
        E_residual_sum = global_add_pool(self.residual_head(edge_out), batch[row])
        
        # 5. Final Energy (BEP Base + Residual + Node Correction)
        # E_fwd = E_BEP + E_Residual
        E_fwd_pred = E_bep_base + E_residual_sum
        
        # [Hess's Law] E_rev = E_fwd - Delta_H
        E_rev_pred = E_fwd_pred - delta_H
        
        return E_fwd_pred, E_rev_pred, late_char

Model

In [16]:
class GeoPINNModel(nn.Module):
    def __init__(self, node_dim=27, hidden_dim=256, num_layers=4):
        super().__init__()
        # 1. Geometric Backbone (SchNet Style RBF)
        self.encoder = GeometricGNN(node_dim, hidden_dim, num_layers)
        
        # 2. Attention Gating
        self.gating = SigmoidGating(hidden_dim)
        
        # 3. Structure & Energy Prediction
        self.struct_decoder = StructureDecoder(hidden_dim)
        self.energy_predictor = PhysChemInformedPredictor(hidden_dim)

    def forward(self, data):
        h = self.encoder(data.x, data.pos, data.edge_index)
        gate_map = self.gating(h, data.edge_index, data.pos)
        pred_delta_D = self.struct_decoder(h, data.edge_index, gate_map)
        
        pred_fwd, pred_rev, ts_char = self.energy_predictor(
            h, data.z, data.hsab, data.edge_index, data.pos, data.pos_P,
            gate_map, pred_delta_D, data.batch, data.thermo_energy, data.edge_bde
        )
        return pred_delta_D, pred_fwd, pred_rev, gate_map, ts_char

학습루프

In [17]:
def train(h5_path, epochs=40, limit=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    
    try: dataset = ReactionDataset(h5_path, limit=limit)
    except NameError: print("Dataset Error."); return None, None, None
        
    print("Computing stats...")
    loader_stat = DataLoader(dataset, batch_size=128, shuffle=False)
    all_E = []
    HARTREE_TO_KCAL = 627.5
    for b in loader_stat: all_E.append(b.y_fwd.view(-1, 3) * HARTREE_TO_KCAL)
    all_E = torch.cat(all_E, dim=0)
    y_mean = all_E.mean(dim=0).to(device); y_std = all_E.std(dim=0).to(device)
    print(f"Stats (Fwd) -> Mean: {y_mean.cpu().numpy()}")

    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    # node_dim=27 (Includes FG info), hidden_dim=256
    model = GeoPINNModel(node_dim=27, hidden_dim=256, num_layers=4).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4)
    
    history = {'loss': []}
    real_E3, pred_E3 = {'H':[], 'G':[], 'E':[]}, {'H':[], 'G':[], 'E':[]}

    print("\n[Start Geometric Physics-Informed Training]")
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch")
        
        for batch in pbar:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            thermo_kcal = batch.thermo_energy.view(-1, 3) * HARTREE_TO_KCAL
            target_fwd = (batch.y_fwd.view(-1, 3) * HARTREE_TO_KCAL - y_mean) / y_std
            target_rev = (batch.y_rev.view(-1, 3) * HARTREE_TO_KCAL - (y_mean - thermo_kcal)) / y_std
            
            # 5 Outputs
            pred_d, pred_fwd, pred_rev, gate, ts_char = model(batch)
            
            # Loss Calculation
            dist_R = (batch.pos[batch.edge_index[0]] - batch.pos[batch.edge_index[1]]).norm(dim=-1, keepdim=True)
            dist_TS = (batch.y_pos[batch.edge_index[0]] - batch.y_pos[batch.edge_index[1]]).norm(dim=-1, keepdim=True)
            
            # 1. Structure Loss (TS geometry prediction)
            l_struct = F.smooth_l1_loss(pred_d, dist_TS - dist_R)
            # 2. Gate Loss (Active bond detection)
            l_gate = F.binary_cross_entropy(gate, (torch.abs(dist_TS - dist_R)>0.1).float())
            # 3. Energy Loss (Accuracy)
            l_fwd = F.smooth_l1_loss(pred_fwd, target_fwd)
            l_rev = F.smooth_l1_loss(pred_rev, target_rev)
            # 4. Hess Law Loss (Consistency: Ea_fwd - Ea_rev = Delta H)
            l_hess = F.mse_loss(pred_fwd - pred_rev, thermo_kcal / y_std)
            
            loss = l_struct + l_gate + l_fwd + 0.5*l_rev + 0.5*l_hess
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_loss += loss.item()
            pbar.set_postfix({'Loss': f"{loss.item():.4f}", 'Hammond': f"{ts_char.mean():.2f}"})
            
            if epoch == epochs - 1:
                r = (batch.y_fwd.view(-1, 3) * HARTREE_TO_KCAL).detach().cpu().numpy()
                p = (pred_fwd.detach().cpu() * y_std.cpu() + y_mean.cpu()).numpy()
                for i, k in enumerate(['H', 'G', 'E']): real_E3[k].extend(r[:, i]); pred_E3[k].extend(p[:, i])
                    
        history['loss'].append(epoch_loss / len(loader))
        scheduler.step(epoch_loss)

    return history, real_E3, pred_E3

In [18]:
if __name__ == "__main__":
    import os
    base_dir = r"C:\Users\ansh1\OneDrive\OneDrive - personal\서류&파일\University of Seoul\연구\Code"
    MY_H5_PATH = os.path.join(base_dir, "RGD1_CHNO.h5")
    
    if os.path.exists(MY_H5_PATH):
        hist, r_E3, p_E3 = train(MY_H5_PATH, epochs=40, limit=10000)
        
        if hist:
            plt.figure(figsize=(16, 5))
            plt.subplot(1, 4, 1); plt.plot(hist['loss'], 'k-'); plt.title("Loss")
            titles, colors = ["Delta H", "Delta G", "Delta E"], ['r', 'g', 'b']
            keys = ['H', 'G', 'E']
            for i, k in enumerate(keys):
                plt.subplot(1, 4, i+2)
                plt.scatter(r_E3[k], p_E3[k], alpha=0.3, color=colors[i], s=5)
                mx = max(max(r_E3[k]), max(p_E3[k]))
                mn = min(min(r_E3[k]), min(p_E3[k]))
                plt.plot([mn, mx], [mn, mx], 'k--')
                plt.title(titles[i])
            plt.tight_layout()
            plt.show()
    else:
        print("파일을 찾을 수 없습니다.")

Device: cpu
Computing stats...


C:\ProgramData\anaconda3\envs\ysl\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Stats (Fwd) -> Mean: [69.403175 74.3915   72.29167 ]

[Start Geometric Physics-Informed Training]


Epoch 1/40:   0%|                                                                           | 0/313 [00:00<?, ?batch/s]C:\Users\ansh1\AppData\Local\Temp\ipykernel_11880\566039350.py:52: UserWarning: Using a target size (torch.Size([32, 3])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  l_fwd = F.smooth_l1_loss(pred_fwd, target_fwd)
C:\Users\ansh1\AppData\Local\Temp\ipykernel_11880\566039350.py:53: UserWarning: Using a target size (torch.Size([32, 3])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  l_rev = F.smooth_l1_loss(pred_rev, target_rev)
C:\Users\ansh1\AppData\Local\Temp\ipykernel_11880\566039350.py:55: UserWarning: Using a target size (torch.Size([32, 3])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect

In [20]:
import numpy as np

def print_evaluation_report(real_dict, pred_dict):
    """
    학습 결과(정답 vs 예측)를 받아서 MAE, RMSE, R2, MSE를 계산하고 출력합니다.
    """
    print("\n" + "="*75)
    print(f"   📊 FINAL PHYSICS-INFORMED MODEL PERFORMANCE REPORT (kcal/mol)")
    print("="*75)

    # 헤더 출력
    header = f"{'Energy Type':<15} | {'MAE':<10} | {'RMSE':<10} | {'MSE':<10} | {'R² Score':<10}"
    print(header)
    print("-" * 75)

    # H, G, E 각각에 대해 계산
    metrics = {}
    for key in ['H', 'G', 'E']:
        y_true = np.array(real_dict[key])
        y_pred = np.array(pred_dict[key])

        # 1. MAE (Mean Absolute Error)
        mae = np.mean(np.abs(y_true - y_pred))

        # 2. MSE (Mean Squared Error)
        mse = np.mean((y_true - y_pred)**2)

        # 3. RMSE (Root Mean Squared Error)
        rmse = np.sqrt(mse)

        # 4. R2 Score (Coefficient of Determination)
        ss_res = np.sum((y_true - y_pred)**2)
        ss_tot = np.sum((y_true - np.mean(y_true))**2)
        r2 = 1 - (ss_res / (ss_tot + 1e-8)) # 0으로 나누기 방지

        # 결과 저장
        metrics[key] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}

        # 행 출력
        label = f"Delta {key} (Δ{key})"
        print(f"{label:<15} | {mae:<10.4f} | {rmse:<10.4f} | {mse:<10.4f} | {r2:<10.4f}")

    print("="*75)

    # [화학적 해석]
    # 화학적 정확도(Chemical Accuracy)는 보통 1 kcal/mol 이내를 말합니다.
    avg_mae = np.mean([metrics[k]['MAE'] for k in ['H', 'G', 'E']])
    print(f"⭐ Average MAE: {avg_mae:.4f} kcal/mol")

    if avg_mae < 2.5:
        print("✅ 모델이 물리 법칙(BEP, Hammond, Pauling)을 매우 잘 학습했습니다!")
        print("   (일반적인 DFT 계산 오차가 2~3 kcal/mol 수준임을 감안하면 훌륭합니다.)")
    elif avg_mae < 5.0:
        print("⚠️ 학습이 잘 되었으나, 일부 복잡한 반응에서 오차가 있습니다.")
    else:
        print("❌ 아직 과소적합 상태이거나 물리적 제약 조건이 더 필요합니다.")
    print("="*75 + "\n")

# ==============================================================================
# [실행 방법] main 블록 안에서 train()이 끝난 후 호출하세요
# ==============================================================================
if __name__ == "__main__":
    # 위에서 이미 train을 실행해서 r_E3, p_E3가 있다고 가정합니다.
    # 만약 변수가 메모리에 없다면 다시 학습을 돌려야 합니다.

    if 'r_E3' in locals() and 'p_E3' in locals():
        print_evaluation_report(r_E3, p_E3)
    else:
        print("먼저 train() 함수를 실행하여 r_E3, p_E3 변수를 생성해주세요.")


   📊 FINAL PHYSICS-INFORMED MODEL PERFORMANCE REPORT (kcal/mol)
Energy Type     | MAE        | RMSE       | MSE        | R² Score  
---------------------------------------------------------------------------
Delta H (ΔH)    | 20.2382    | 25.4194    | 646.1454   | 0.1874    
Delta G (ΔG)    | 19.9909    | 24.9862    | 624.3090   | 0.1149    
Delta E (ΔE)    | 20.8788    | 26.2641    | 689.8038   | 0.1962    
⭐ Average MAE: 20.3693 kcal/mol
❌ 아직 과소적합 상태이거나 물리적 제약 조건이 더 필요합니다.

